# LLMChain with LCEL

In [3]:
# LLM Model
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate

OPEN_AI_APIKEY = "{YOUR_OPENAI_KEY}"
model = ChatOpenAI(api_key=OPEN_AI_APIKEY)

prompt = PromptTemplate.from_template("what is the famous tour place in {city}?")
chain = prompt | model
city = "Seoul"

response1 = chain.invoke({"city": city})
response2 = chain.invoke(city)
print(response1)
print(response2)

content="Seoul, the capital of South Korea, is home to many famous tourist attractions. Some of the most popular places to visit include:\n\n1. **Gyeongbokgung Palace**: The largest of the Five Grand Palaces built during the Joseon Dynasty, it features beautiful architecture and gardens.\n\n2. **Bukchon Hanok Village**: A traditional Korean village with well-preserved hanok (traditional Korean houses), offering a glimpse into the country's past.\n\n3. **N Seoul Tower (Namsan Tower)**: An iconic landmark that provides panoramic views of the city, especially beautiful at night.\n\n4. **Myeongdong**: A bustling shopping district known for its street food, cosmetics shops, and fashion boutiques.\n\n5. **Insadong**: A cultural district famous for its art galleries, traditional tea houses, and shops selling antiques and crafts.\n\n6. **Dongdaemun Design Plaza (DDP)**: A modern architectural marvel that hosts exhibitions, fashion shows, and markets.\n\n7. **Changdeokgung Palace and Secret Gar

# Sequential Chain with LCEL
ref : https://python.langchain.com/docs/expression_language/cookbook/multiple_chains

In [4]:
# Sequential Chain with LCEL
from operator import itemgetter

from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate

OPEN_AI_APIKEY = "{YOUR_OPENAI_KEY}"
model = ChatOpenAI(api_key=OPEN_AI_APIKEY)

prompt1 = PromptTemplate.from_template(
    "what is the famous tour place in {city}? Tell me the name of the place only without additional comments."
)
prompt2 = PromptTemplate.from_template("How can I get {place} by {transport}?")
chain1 = prompt1 | model
chain2 = prompt2 | model
chain = {"place": chain1, "transport": itemgetter("transport")} | chain2
chain.invoke({"city": "Seoul", "transport": "subway"})

AIMessage(content='To get to Gyeongbokgung Palace by subway in Seoul, follow these steps:\n\n1. **Find the Nearest Subway Station**: The closest subway station to Gyeongbokgung Palace is **Gyeongbokgung Station** on Line 3 (the Orange Line).\n\n2. **Directions to Gyeongbokgung Station**:\n   - If you are starting from **Incheon International Airport**, take the Airport Railroad Express (AREX) to **Seoul Station**. From Seoul Station, transfer to Line 4 (the Blue Line) and go to **Gwanghwamun Station**. From there, you can walk to Gyeongbokgung Palace.\n   - If you are starting from **Seoul Station**, take Line 1 (the Dark Blue Line) to **City Hall Station**, then transfer to Line 2 (the Green Line) and go to **Gyeongbokgung Station**.\n\n3. **Exit the Station**: Once you arrive at Gyeongbokgung Station, use Exit 5. The palace is a short walk from the station.\n\n4. **Walking Directions**: After exiting, follow the signs to Gyeongbokgung Palace. It should take about 5-10 minutes on foot

# Sequential Advacned with LCEL

In [5]:
from operator import itemgetter
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

OPEN_AI_APIKEY = "{YOUR_OPENAI_KEY}"
model = ChatOpenAI(api_key=OPEN_AI_APIKEY)

prompt1 = PromptTemplate.from_template(
    "what is the famous tour place in {city}? Tell me the name of the place only without additional comments."
)
prompt2 = PromptTemplate.from_template(
    "What is the top 5 restaurant in the {place} in city {city} without additional comments?"
)  # output : restaurants
prompt3 = PromptTemplate.from_template(
    "What is the best one restaurant and food for family dinner among {restaurants} ?"
)  # output : restaurant_information
prompt4 = PromptTemplate.from_template(
    "How can I get the {place} by using {transport}?"
)  # output : transport_information
final_prompt = PromptTemplate.from_template(
    """
Please summarize the tour information with reastaurant information and transportation by using the this information.
Restaurant informations : {restaurant_information}
Transport information : {transport_information}
"""
)

chain1 = {"city": itemgetter("city")} | prompt1 | model | StrOutputParser()
chain2 = (
    {"place": chain1, "city": itemgetter("city")} | prompt2 | model | StrOutputParser()
)
chain3 = {"restaurants": chain2} | prompt3 | model | StrOutputParser()
chain4 = (
    {"place": chain1, "transport": itemgetter("transport")}
    | prompt4
    | model
    | StrOutputParser()
)
final_chain = (
    {"restaurant_information": chain3, "transport_information": chain4}
    | final_prompt
    | model
    | StrOutputParser()
)

final_chain.invoke({"city": "Seoul", "transport": "subway"})

'### Tour Information Summary\n\n**Dining Options:**\nWhen choosing a restaurant for a family dinner, consider the following options based on your preferences:\n\n1. **Tosokchon Samgyetang**: Ideal for families seeking a traditional Korean experience, famous for its ginseng chicken soup in a casual atmosphere.\n\n2. **The Lounge & Bar at The Four Seasons Hotel Seoul**: Offers a luxurious dining experience with a diverse international menu, suitable for families looking for upscale dining.\n\n3. **Hanok Jib**: Provides a traditional Korean dining experience in a hanok setting, family-friendly with a variety of local dishes.\n\n4. **Bukchon Son Mandu**: A cozy spot known for handmade dumplings, perfect for casual dining and popular with kids.\n\n5. **Café de Flore**: A relaxed café offering a mix of Western and Korean dishes, suitable for families wanting a casual dining atmosphere.\n\n**Recommendation**: For a traditional Korean meal in a casual setting, **Tosokchon Samgyetang** is high

# Router with LCEL


In [6]:
from operator import itemgetter
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableBranch

# OPEN_AI_APIKEY = "{YOUR_OPENAI_KEY}"
# model = ChatOpenAI(api_key=OPEN_AI_APIKEY)

restaurant_template = """
You are a tourist guide. You know many good restaurants around the tourist destination.
You can recommend good foods and restaurants.
Here is a question:
{input}
"""

transport_template = """
You are a tourist guide. You have a lot of knowledge in public transportation.
You can provide information about public transportation to help tourists get to the tourist destination.
Here is a question:
{input}
"""

destination_template = """
You are a tourist guide. You know many good tourist places.
You can recommend good tourist places to the tourists.
Here is a question:
{input}
"""

prompt_infos = [
    {
        "name": "restaurants",
        "description": "Good for recommending restaurants around the tourist destinations",
        "prompt_template": restaurant_template,
    },
    {
        "name": "transport",
        "description": "Good for guiding the transport to get the place",
        "prompt_template": transport_template,
    },
    {
        "name": "destination",
        "description": "Good for recommending place to tour",
        "prompt_template": destination_template,
    },
]

destination_chains = {}
for prompt_info in prompt_infos:
    name = prompt_info["name"]
    prompt = PromptTemplate.from_template(prompt_info["prompt_template"])
    chain = prompt | model | StrOutputParser()
    destination_chains[name] = chain

destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

default_prompt = PromptTemplate.from_template("{input}")
default_chain = default_prompt | model | StrOutputParser()

router_prompt = PromptTemplate(
    template="""
For the given question, refer to the explanations for the following categories below and select the appropriate category. 
If there is no suitable category, return the string "default".
The output should be single word 

Categories are composed of strings in the format "Category: Description", and the categories are as follows:
{destinations_str}

Question:
{input}
""",
    input_variables=["input"],
    partial_variables={"destinations_str": destinations_str},
)

print(destinations_str)
router_chain = (
    {"input": itemgetter("input")} | router_prompt | model | StrOutputParser()
)

# branch = RunnableBranch(
#    (lambda x: "restaurants" in x['destination'].lower(), destination_chains['restaurants']),
#    (lambda x: "transport" in x['destination'].lower(), destination_chains['transport']),
#    (lambda x: "destination" in x['destination'].lower(), destination_chains['destination']),
#    default_chain,
# )

branch = RunnableBranch(
    *(
        (
            lambda x, name=prompt_info["name"], chain=destination_chains[name]: name
            in x["destination"].replace("\n", ""),
            chain,
        )
        for prompt_info in prompt_infos
    ),
    default_chain,
)

full_chain = {"input": itemgetter("input"), "destination": router_chain} | branch

input = "What is best restaurant in Seoul?"
output = full_chain.invoke({"input": input})

output

restaurants: Good for recommending restaurants around the tourist destinations
transport: Good for guiding the transport to get the place
destination: Good for recommending place to tour


"Seoul is known for its vibrant food scene, offering a wide range of dining options. Here are some of the best restaurants you might want to consider:\n\n1. **Mingles** - A Michelin-starred restaurant that blends traditional Korean flavors with modern techniques. The seasonal tasting menu is highly recommended.\n\n2. **Jungsik** - Another Michelin-starred establishment, Jungsik offers a contemporary take on Korean cuisine, with beautifully presented dishes and a sophisticated atmosphere.\n\n3. **Gwangjang Market** - While not a restaurant per se, this market is a must-visit for street food lovers. Try the bindaetteok (mung bean pancakes) and mayak gimbap (addictive rice rolls).\n\n4. **Tosokchon Samgyetang** - Famous for its ginseng chicken soup, this restaurant is a favorite among locals and tourists alike. The rich broth and tender chicken make it a comforting choice.\n\n5. **Baekjeong** - A popular Korean BBQ chain known for its high-quality meats and lively atmosphere. It's a great